In [43]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import plotly.graph_objects as go
import sys
sys.path.append('../../assets/python/')
import tfb

METADATA = {'Contributor': 'T. Dunn'}
FSIZE = [5,4]
FDPI = 100
SAVEFIGS = False

In [12]:
# used for diagrams
def adjusted_XP_total(XP_log):
    return np.sum([x['XP'] for x in XP_log])

def damage_overflow_total(XP_log):
    dmg = 0
    for x in XP_log:
        if x['target'] == x['ally']:
            dmg += x['damage'] - x['eHP_delta']
    return dmg

def damage_total(XP_log):
    dmg = 0
    for x in XP_log:
        if x['target'] == x['ally']:
            dmg += x['eHP_delta']
    return dmg

def encounter_XP(eHPs, eDPRs):
    # calculates the XP total for the encounter
    return 0.25*np.dot(eHPs, eDPRs)

def encounter_XP_total(XP_grid, weight_grid):
    return np.sum(np.multiply(weight_grid, XP_grid))

In [20]:
# used for graphs
def encounter_multiplier_approx(pc_count=4, npc_count=1, pc_deaths=0, aoe_pct=0, aoe_eff=1.0):
    """Returns the approximate encounter multiplier
    pc_count -- number of PCs in the encounter
    npc_count -- number of NPCs in the encounter
    pc_deaths -- number of PCs expected to die during encounter
    aoe_pct -- decimal percent of damage done to each NPC using AoE effects
    AoE_eff -- ratio of AoE DPR per target to single target DPR
    """
    pd = min(pc_deaths, pc_count)
    em = (npc_count + 1)*(1 - aoe_pct)/2 + aoe_pct/aoe_eff
    em *= 8.0*pc_count/(2*pc_count*pc_count - 2*pd*pc_count + pd*(pd + 1))
    return em

def encounter_multiplier_DMG(pc_count, npc_count):
    """Returns the encounter multiplier given by the DMG
    pc_count -- number of PCs in the encounter
    npc_count -- number of NPCs in the encounter
    """
    n_array = np.asarray([1,2,3,7,11,15])
    m_array = np.asarray([0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 4.0, 5.0])
    i = 1 + n_array[n_array <= max(npc_count,1)].argmax()
    if pc_count >= 6:
        i -= 1
    elif pc_count <= 2:
        i += 1
    return m_array[i]

In [2]:
# XP log functions
def create_XP_log(eHPs, eDPRs, damage):
    """Returns an XP_log
    eHPs -- list of effective hit point values
    eDPRs -- list of effective damage per round values
    damage -- list of damage entries
        damage = [
            {'targets': [1], 'damage': 5, 'efficiency': 1.0, 'color': 'w'},
            {'targets': [0], 'damage': 3, 'efficiency': 1.0, 'color': 'w'}
        ]
    """
    nC = len(eDPRs)
    eDPR_mins = [sum(eDPRs[0:i]) for i in range(nC)]
    eHP_mins = [sum(eHPs[0:i]) for i in range(nC)]
    dmgs = [0]*nC
    eHP_dmgs = [0]*nC
    XP_log = []
    for d in damage:
        # determine if the damage affects a single target or multiple
        if len(d['targets']) > 1:
            dmg_type = 'multiple'
        else:
            dmg_type = 'single'

        # determine which target takes the most damage
        for i in range(nC):
            if (i in d['targets']):
                dmgs[i] = min(eHP_dmgs[i] + d['damage'], eHPs[i]) - eHP_dmgs[i]
            else:
                dmgs[i] = 0
        t_max = np.argmax(dmgs)
        dmg_max = dmgs[t_max]

        # construct log entries for damage
        for i in range(nC):
            if eHP_dmgs[i] < eHPs[i]:
                if i not in d['targets']:
                    dmg = dmg_max
                    t = t_max
                else:
                    dmg = dmgs[i]
                    t = i
                
                XP_log.append({
                    'type': dmg_type,
                    'efficiency': d['efficiency'],
                    'target': t,
                    'ally': i,
                    'damage': d['damage'],
                    'XP': 0.25*dmg*eDPRs[i]/(d['efficiency']),
                    'eHP_min': eHP_mins[t] + eHP_dmgs[t],
                    'eDPR_min': eDPR_mins[i],
                    'eHP_delta': dmg,
                    'eDPR_delta': eDPRs[i],
                    'color': d['color']
                })
        
        # update damage taken
        for t in d['targets']:
            eHP_dmgs[t] += dmgs[t]
        
    return XP_log

def generate_XP_diagram_plotly(eHPs, eDPRs, XP_log, **kwargs):
    """Generate XP diagram and returns axes for that diagram
    eHPs -- list of effective hit point values
    eDPRs -- list of effective damage per round values
    XP_log -- list of XP contributions for the encounter
    **kwargs
        fontsize -- font size for the figure's text
        ax -- pyplot axes
    """
    nC = len(eHPs)
    eHP_mins = [sum(eHPs[0:i]) for i in range(len(eHPs))]
    eDPR_mins = [sum(eDPRs[0:i]) for i in range(len(eDPRs))]
    eHP_max = sum(eHPs)
    eDPR_max = sum(eDPRs)

    fontsize = kwargs.get('fontsize', None)
    fig = kwargs.get('fig', go.Figure())
    title = kwargs.get('title', None)

    # initialize figure
    #ax.add_patch(plt.Rectangle((0,0), eDPR_max, eHP_max, fc='white', ec='none', alpha=1.0))

    # apply damage and color XP boxes accordingly
    for x in XP_log:
        fig.add_shape(type="rect",
            x0=x['eDPR_min'], x1=x['eDPR_min']+x['eDPR_delta'], 
            y0=x['eHP_min'],  y1=x['eHP_min']+x['eHP_delta'],
            line=dict(color='rgba(250,250,250,0)'),
            fillcolor=x['color'],
            opacity=0.3,
        )

    # add boarder lines
    def add_reference_line(fig, x, y):
        fig.add_trace(go.Scatter(
            x=x, y=y,
            mode='lines',
            line=dict(color='black', dash='solid'),
            showlegend=False, 
            hoverinfo='skip',
        ))

    for i in range(nC):
        add_reference_line(fig, x=[0, eDPR_max], y=[eHP_mins[i],eHP_mins[i]])
        add_reference_line(fig, x=[eDPR_mins[i],eDPR_mins[i]], y=[0, eHP_max])
    add_reference_line(fig, x=[0, eDPR_max], y=[eHP_max, eHP_max])
    add_reference_line(fig, x=[eDPR_max, eDPR_max], y=[0, eHP_max])

    # create XP box borders
    for i in range(nC):
        for j in range(nC):
            if i == j:
                ls = '-'
            else:
                ls = '-'
            #ax.add_patch(plt.Rectangle((eDPR_mins[i], eHP_mins[j]), eDPRs[i], eHPs[j], fc='none', ec='black', linestyle=ls))

    # create XP box labels
    for i in range(nC):
        for j in range(nC):
            fig.add_annotation(
                x=eDPR_mins[i] + 0.5*eDPRs[i], 
                y=eHP_mins[j] + 0.5*eHPs[j],
                showarrow=False,
                text=f'$\mathit{{XP}}_{{{j+1},{i+1}}}$',
            )

    # create axes labels
    for i in range(nC):
        # x-axis
        fig.add_annotation(
            x=eDPR_mins[i] + 0.5*eDPRs[i], y=-(0.7/16)*eHP_max, 
            showarrow=False,
            text=f'$\mathit{{eDPR}}_{{{i+1}}}$',
        )
        # y-axis
        fig.add_annotation(
            x=-(0.7/16)*eDPR_max, y=(eHP_mins[i] + 0.5*eHPs[i]), 
            showarrow=False,
            text=f'$\mathit{{eHP}}_{{{i+1}}}$',
            textangle=-90,
        )

    return fig

In [4]:
AoE_eff = 0.7
AoE_pct = [0.2,0.2,0.6,0.6,0.6]
CR = [5,5,2,2,2]
XP = [CR_to_XP(c) for c in CR]
XP_grid = encounter_XP_grid_approx(XP)
w_grid = encounter_weight_grid_mixed_random(XP, AoE_pct, AoE_eff)
XPtotal = XP_total(XP)
eXPtotal = encounter_XP_total(XP_grid, w_grid)

print('Method: {}; XP total = {:.0f}; encounter XP total = {:.0f}; encounter multiplier = {:.1f}'.format('mixed', XPtotal, eXPtotal, eXPtotal/XPtotal))

eXPtotal = encounter_XP_total_DMG(XP)
print('Method: {}; XP total = {:.0f}; encounter XP total = {:.0f}; encounter multiplier = {:.1f}'.format('DMG', XPtotal, eXPtotal, eXPtotal/XPtotal))


Method: mixed; XP total = 4950; encounter XP total = 6930; encounter multiplier = 1.4
Method: DMG; XP total = 4950; encounter XP total = 9900; encounter multiplier = 2.0


In [46]:
# An example encounter diagram for two NPCs
eDPRs = [4, 4]
eHPs  = [3, 5]
eDPR_total = np.sum(eDPRs)
eHP_total = np.sum(eHPs)

damage = [
    {'targets': [1], 'damage': 5, 'efficiency': 1.0, 'color': 'white'},
    {'targets': [0], 'damage': 3, 'efficiency': 1.0, 'color': 'white'}
]

XP_log = create_XP_log(eHPs, eDPRs, damage)

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=15, r=15, b=15, t=15, pad=4),
        #margin=dict(autoexpand=True),
        dragmode=False,
        xaxis=dict(
            range=[-0.1*eDPR_total, 1.1*eDPR_total],
            #autorange=True,
            mirror=False,
            showgrid=False,
            showline=False,
            zeroline=False,
            showticklabels=False,
            ticks='', minor=dict(ticks=''),
        ),
        yaxis=dict(
            range=[-0.1*eHP_total, 1.1*eHP_total],
            #autorange=True,
            mirror=False,
            showgrid=False,
            showline=False,
            zeroline=False,
            showticklabels=False,
            ticks='', minor=dict(ticks=''),
        ),
    )
)

fig = generate_XP_diagram_plotly(eHPs, eDPRs, XP_log, fig=fig)

# show figure
fig.update_layout(width=600, height=600)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name='fig-xp-encounter-diagram-two-npcs-large', style='aspect-ratio: 600/600;')
    tfb.save_fig_html(fig, format='small', name='fig-xp-encounter-diagram-two-npcs-small', style='aspect-ratio: 600/600;')

enc_XP_total = encounter_XP(eDPRs, eHPs)
adj_XP_total = adjusted_XP_total(XP_log)
print('encounter multiplier: {:.2f}'.format(adj_XP_total/enc_XP_total))
eHP_total = sum(eHPs)
dmg_total = damage_total(XP_log)
dmg_overflow = damage_overflow_total(XP_log)
print('percent damage done: {} %'.format(100*dmg_total/eHP_total))
print('percent damage overflow: {} %'.format(100*(dmg_overflow/eHP_total)))


encounter multiplier: 1.62
percent damage done: 100.0 %
percent damage overflow: 0.0 %


In [15]:
# An example encounter diagram for two NPCs defeated in reverse order using single target damage
eDPRs = [4, 4]
eHPs  = [3, 5]
eDPR_total = np.sum(eDPRs)
eHP_total = np.sum(eHPs)

damage = [
    {'targets': [1], 'damage': 5, 'efficiency': 1.0, 'color': 'blue'},
    {'targets': [0], 'damage': 3, 'efficiency': 1.0, 'color': 'blue'}
]

XP_log = create_XP_log(eHPs, eDPRs, damage)

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=15, r=15, b=15, t=15, pad=4),
        title=dict(
            text=r'$\mathrm{NPC}\ 2 \rightarrow \mathrm{NPC}\ 1$',
            font_size=16,
            x=0.5,
            y=0.95,
            xanchor='center',
            yanchor='top',
        ),
        #margin=dict(autoexpand=True),
        dragmode=False,
        xaxis=dict(
            range=[-0.1*eDPR_total, 1.1*eDPR_total],
            #autorange=True,
            mirror=False,
            showgrid=False,
            showline=False,
            zeroline=False,
            showticklabels=False,
            ticks='', minor=dict(ticks=''),
        ),
        yaxis=dict(
            range=[-0.1*eHP_total, 1.1*eHP_total],
            #autorange=True,
            mirror=False,
            showgrid=False,
            showline=False,
            zeroline=False,
            showticklabels=False,
            ticks='', minor=dict(ticks=''),
        ),
    )
)

fig = generate_XP_diagram_plotly(eHPs, eDPRs, XP_log, fig=fig)

#fig = plt.figure(figsize=FSIZE, dpi=FDPI, tight_layout=True)
#ax = plt.axes()
#ax = generate_XP_diagram(ax, eHPs, eDPRs, XP_log, r'NPC 2 $\rightarrow$ NPC 1', fontsize=12)

#if SAVEFIGS: fig.savefig('./fig-xp-encounter-diagram-two-npcs-reverse.svg', metadata=METADATA)

# show figure
fig.update_layout(width=600, height=600)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name='fig-xp-encounter-diagram-two-npcs-reverse-large', style='aspect-ratio: 600/600;')
    tfb.save_fig_html(fig, format='small', name='fig-xp-encounter-diagram-two-npcs-reverse-small', style='aspect-ratio: 600/600;')

enc_XP_total = encounter_XP(eDPRs, eHPs)
adj_XP_total = adjusted_XP_total(XP_log)
print('encounter multiplier: {:.2f}'.format(adj_XP_total/enc_XP_total))
eHP_total = sum(eHPs)
dmg_total = damage_total(XP_log)
dmg_overflow = damage_overflow_total(XP_log)
print('percent damage done: {} %'.format(100*dmg_total/eHP_total))
print('percent damage overflow: {} %'.format(100*(dmg_overflow/eHP_total)))

encounter multiplier: 1.62
percent damage done: 100.0 %
percent damage overflow: 0.0 %


In [16]:
# An example encounter diagram for two NPCs defeated in order using single target damage
eDPRs = [4, 4]
eHPs  = [3, 5]
eDPR_total = np.sum(eDPRs)
eHP_total = np.sum(eHPs)

damage = [
    {'targets': [0], 'damage': 3, 'efficiency': 1.0, 'color': 'blue'},
    {'targets': [1], 'damage': 5, 'efficiency': 1.0, 'color': 'blue'}
]

XP_log = create_XP_log(eHPs, eDPRs, damage)

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=15, r=15, b=15, t=15, pad=4),
        title=dict(
            text=r'$\mathrm{NPC}\ 1 \rightarrow \mathrm{NPC}\ 2$',
            font_size=16,
            x=0.5,
            y=0.95,
            xanchor='center',
            yanchor='top',
        ),
        #margin=dict(autoexpand=True),
        dragmode=False,
        xaxis=dict(
            range=[-0.1*eDPR_total, 1.1*eDPR_total],
            #autorange=True,
            mirror=False,
            showgrid=False,
            showline=False,
            zeroline=False,
            showticklabels=False,
            ticks='', minor=dict(ticks=''),
        ),
        yaxis=dict(
            range=[-0.1*eHP_total, 1.1*eHP_total],
            #autorange=True,
            mirror=False,
            showgrid=False,
            showline=False,
            zeroline=False,
            showticklabels=False,
            ticks='', minor=dict(ticks=''),
        ),
    )
)

fig = generate_XP_diagram_plotly(eHPs, eDPRs, XP_log, fig=fig)

# show figure
fig.update_layout(width=600, height=600)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name='fig-xp-encounter-diagram-two-npcs-forward-large', style='aspect-ratio: 600/600;')
    tfb.save_fig_html(fig, format='small', name='fig-xp-encounter-diagram-two-npcs-forward-small', style='aspect-ratio: 600/600;')

enc_XP_total = encounter_XP(eDPRs, eHPs)
adj_XP_total = adjusted_XP_total(XP_log)
print('encounter multiplier: {:.2f}'.format(adj_XP_total/enc_XP_total))
eHP_total = sum(eHPs)
dmg_total = damage_total(XP_log)
dmg_overflow = damage_overflow_total(XP_log)
print('percent damage done: {} %'.format(100*dmg_total/eHP_total))
print('percent damage overflow: {} %'.format(100*(dmg_overflow/eHP_total)))

encounter multiplier: 1.38
percent damage done: 100.0 %
percent damage overflow: 0.0 %


In [21]:
# Single target encounter multiplier as a function of the number of NPCs.

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=60, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='number of NPCs',
            range=[0,20],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='encounter multiplier',
            range=[0,11],
            tick0=0, dtick=2,
            minor=dict(tick0=0, dtick=1),
            #tickformat = '.0f',
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

# single target
nCs = list(range(1, 21))
fig.add_trace(go.Scatter(
    x=nCs,
    y=[encounter_multiplier_approx(pc_count=4, npc_count=n) for n in nCs],
    mode='lines',
    name='single target',
    hovertemplate=
        'NPCs %{x:d}<br>'+
        'multiplier %{y:,.1f}' + 
        '<extra></extra>'
))

# DMG
nCs = list(range(1, 21))
fig.add_trace(go.Scatter(
    x=nCs,
    y=[encounter_multiplier_DMG(4, n) for n in nCs],
    mode='lines',
    name='DMG',
    hovertemplate=
        'NPCs %{x:d}<br>'+
        'multiplier %{y:,.1f}' + 
        '<extra></extra>'
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name='fig-encounter-approximate-general-single-target-large')
    tfb.save_fig_html(fig, format='small', name='fig-encounter-approximate-general-single-target-small')


In [22]:
# An example encounter diagram for two NPCs defeated in order using single target damage
eDPRs = [4, 4]
eHPs  = [3, 5]
eDPR_total = np.sum(eDPRs)
eHP_total = np.sum(eHPs)

damage = [
    {'targets': [0,1], 'damage': 5, 'efficiency': 1.0, 'color': 'red'}
]

XP_log = create_XP_log(eHPs, eDPRs, damage)

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=15, r=15, b=15, t=15, pad=4),
        #margin=dict(autoexpand=True),
        dragmode=False,
        xaxis=dict(
            range=[-0.1*eDPR_total, 1.1*eDPR_total],
            #autorange=True,
            mirror=False,
            showgrid=False,
            showline=False,
            zeroline=False,
            showticklabels=False,
            ticks='', minor=dict(ticks=''),
        ),
        yaxis=dict(
            range=[-0.1*eHP_total, 1.1*eHP_total],
            #autorange=True,
            mirror=False,
            showgrid=False,
            showline=False,
            zeroline=False,
            showticklabels=False,
            ticks='', minor=dict(ticks=''),
        ),
    )
)

fig = generate_XP_diagram_plotly(eHPs, eDPRs, XP_log, fig=fig)

# show figure
fig.update_layout(width=600, height=600)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name='fig-xp-encounter-diagram-two-npcs-multi-uniform-large', style='aspect-ratio: 600/600;')
    tfb.save_fig_html(fig, format='small', name='fig-xp-encounter-diagram-two-npcs-multi-uniform-small', style='aspect-ratio: 600/600;')

enc_XP_total = encounter_XP(eDPRs, eHPs)
adj_XP_total = adjusted_XP_total(XP_log)
print('encounter multiplier: {:.2f}'.format(adj_XP_total/enc_XP_total))
eHP_total = sum(eHPs)
dmg_total = damage_total(XP_log)
dmg_overflow = damage_overflow_total(XP_log)
print('percent damage done: {} %'.format(100*dmg_total/eHP_total))
print('percent damage overflow: {} %'.format(100*(dmg_overflow/eHP_total)))


encounter multiplier: 1.00
percent damage done: 100.0 %
percent damage overflow: 25.0 %


In [23]:
# Multi-target encounter multiplier as a function of multi-target damage efficiency.
# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=60, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='multi-target damage efficiency',
            range=[0.0,1.0],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        yaxis=dict(
            title_text='encounter multiplier',
            range=[0,10],
            tick0=0, dtick=2,
            minor=dict(tick0=0, dtick=1),
        ),
    )
)

eff = np.linspace(0.1, 1.0, 91)
em = [encounter_multiplier_approx(pc_count=4, npc_count=1, pc_deaths=0, aoe_pct=1.0, aoe_eff=x) for x in eff]
fig.add_trace(go.Scatter(
    x=eff, 
    y=em,
    mode='lines', 
    hovertemplate=
        'MT efficiency %{x:.2f}<br>'+
        'multiplier %{y:,.1f}' + 
        '<extra></extra>'
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name='fig-encounter-multiplier-general-multi-target-large')
    tfb.save_fig_html(fig, format='small', name='fig-encounter-multiplier-general-multi-target-small')

In [24]:
# An example encounter diagram for two NPCs defeated in order using single target damage
eDPRs = [4, 4]
eHPs  = [3, 5]
eDPR_total = np.sum(eDPRs)
eHP_total = np.sum(eHPs)

damage = [
    {'targets': [0,1], 'damage': 2, 'efficiency': 1.0, 'color': 'red'},
    {'targets': [0], 'damage': 1, 'efficiency': 1.0, 'color': 'blue'},
    {'targets': [1], 'damage': 3, 'efficiency': 1.0, 'color': 'blue'}
]

XP_log = create_XP_log(eHPs, eDPRs, damage)

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=15, r=15, b=15, t=15, pad=4),
        #margin=dict(autoexpand=True),
        dragmode=False,
        xaxis=dict(
            range=[-0.1*eDPR_total, 1.1*eDPR_total],
            #autorange=True,
            mirror=False,
            showgrid=False,
            showline=False,
            zeroline=False,
            showticklabels=False,
            ticks='', minor=dict(ticks=''),
        ),
        yaxis=dict(
            range=[-0.1*eHP_total, 1.1*eHP_total],
            #autorange=True,
            mirror=False,
            showgrid=False,
            showline=False,
            zeroline=False,
            showticklabels=False,
            ticks='', minor=dict(ticks=''),
        ),
    )
)

fig = generate_XP_diagram_plotly(eHPs, eDPRs, XP_log, fig=fig)

# show figure
fig.update_layout(width=600, height=600)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name='fig-xp-encounter-diagram-two-npcs-mixed-forward-large', style='aspect-ratio: 600/600;')
    tfb.save_fig_html(fig, format='small', name='fig-xp-encounter-diagram-two-npcs-mixed-forward-small', style='aspect-ratio: 600/600;')


enc_XP_total = encounter_XP(eDPRs, eHPs)
adj_XP_total = adjusted_XP_total(XP_log)
print('encounter multiplier: {:.2f}'.format(adj_XP_total/enc_XP_total))
eHP_total = sum(eHPs)
dmg_total = damage_total(XP_log)
dmg_overflow = damage_overflow_total(XP_log)
print('percent damage done: {} %'.format(100*dmg_total/eHP_total))
print('percent damage overflow: {} %'.format(100*(dmg_overflow/eHP_total)))

encounter multiplier: 1.12
percent damage done: 100.0 %
percent damage overflow: 0.0 %


In [22]:
# Plots the encounter multiplier as a function of the number of enemy NPCs.
# All NPCs are assumed to be identical. AoE effects are assumed to damage 
# all NPCs equally.

groups = {
    'single target': {
        'AoE pct': 0.00,
        'AoE eff': 0.70
    },
    '25% MT damage': {
        'AoE pct': 0.25,
        'AoE eff': 0.70
    },
    '50% MT damage': {
        'AoE pct': 0.50,
        'AoE eff': 0.70
    },
    '75% MT damage': {
        'AoE pct': 0.75,
        'AoE eff': 0.70
    },
    'all MT damage': {
        'AoE pct': 1.00,
        'AoE eff': 0.70
    }
}

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=60, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='NPCs',
            range=[0,21],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='encounter multiplier',
            range=[0,10],
            tick0=0, dtick=2,
            minor=dict(tick0=0, dtick=1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

n_max = 20
npcs = list(range(1, n_max+1))
# groups
for group in groups:
    AoE_pct = groups[group]['AoE pct']
    AoE_eff = groups[group]['AoE eff']
    fig.add_trace(go.Scatter(
        x=npcs, 
        y=[encounter_multiplier_approx(pc_count=4, npc_count=n, pc_deaths=0, aoe_pct=AoE_pct, aoe_eff=AoE_eff) for n in npcs],
        mode='lines', 
        name=group,
        hovertemplate=
            f'<b>{group}</b><br>' + 
            'NPCs %{x:d}<br>'+
            'multiplier %{y:,.1f}' + 
            '<extra></extra>'
    ))

# DMG
group = 'DMG'
fig.add_trace(go.Scatter(
    x=npcs, 
    y=[encounter_multiplier_DMG(4, n) for n in npcs],
    mode='lines', 
    name=group,
    line_color='black',
    hovertemplate=
        f'<b>{group}</b><br>' + 
        'NPCs %{x:d}<br>'+
        'multiplier %{y:,.1f}' + 
        '<extra></extra>'
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name='fig-encounter-multiplier-approx-mixed-target-large')
    tfb.save_fig_html(fig, format='small', name='fig-encounter-multiplier-approx-mixed-target-small')

In [38]:
# Plots the encounter multiplier as a function of the number of enemy NPCs.
# All NPCs are assumed to be identical. AoE effects are assumed to damage 
# all NPCs equally.
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(
    specs=[[{'secondary_y': True}]]
)
fig.update_layout(
    template=tfb.FIG_TEMPLATE,
    margin=dict(l=60, r=25, b=55, t=20, pad=4),
    xaxis=dict(
        title_text='NPCs',
        range=[0,21],
        tick0=0, dtick=5,
        minor=dict(tick0=0, dtick=1),
    ),
    yaxis=dict(
        title_text='encounter multiplier',
        range=[0,5],
        tick0=0, dtick=1,
        minor=dict(tick0=0, dtick=0.5),
    ),
    legend=dict(
        xanchor='left', yanchor='top',
        x=0.00, y=1.00,
        orientation='v',
        tracegroupgap=0,
    )
)
fig.update_yaxes(
    secondary_y=True,
    title_text="AoE percent", 
    range=[0,1],
    tick0=0, dtick=0.2,
    minor=dict(tick0=0, dtick=0.1),
)

AoE_eff = 0.70
n_max = 20

# DMG
x = range(1, n_max+1)
npcs = list(range(1, n_max+1))
group = 'DMG'
fig.add_trace(go.Scatter(
    x=npcs, 
    y=[encounter_multiplier_DMG(4, n) for n in npcs],
    mode='lines', 
    name=group,
    line_color='black',
    hovertemplate=
        f'<b>{group}</b><br>' + 
        'NPCs %{x:d}<br>'+
        'multiplier %{y:,.1f}' + 
        '<extra></extra>'
))

# fit
group = 'fit'
npcs     = [1, 2, 3,    5,    7,    9,   11,   13,   15,   18,   20] # number of creatures
AoE_pcts = [0, 0, 0, 0.50, 0.59, 0.63, 0.65, 0.67, 0.68, 0.69, 0.70] # AoE damage percentages
fig.add_trace(go.Scatter(
    x=npcs, 
    y=[encounter_multiplier_approx(pc_count=4, npc_count=npcs[i], pc_deaths=0, aoe_pct=AoE_pcts[i], aoe_eff=AoE_eff) for i in range(len(npcs))],
    mode='lines', 
    name=group,
    line_color='blue',
    line_dash='dash',
    hovertemplate=
        f'<b>{group}</b><br>' + 
        'NPCs %{x:d}<br>'+
        'multiplier %{y:,.1f}' + 
        '<extra></extra>'
))

# fit - AoE percentage
group = 'fit - AoE percent'
fig.add_trace(go.Scatter(
    x=npcs, 
    y=AoE_pcts,
    mode='lines', 
    name=group,
    line_color='red',
    hovertemplate=
        f'<b>{group}</b><br>' + 
        'NPCs %{x:d}<br>'+
        '%{y:,.1%} AoE' + 
        '<extra></extra>'
    ),
    secondary_y=True,
)

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name='fig-encounter-multiplier-identical-npcs-large')
    #tfb.save_fig_html(fig, format='small', name='fig-encounter-multiplier-identical-npcs-small')

In [25]:
# Plots the encounter multiplier as a function of the number of enemy NPCs.
# All NPCs are assumed to be identical. AoE effects are assumed to damage 
# all NPCs equally.

dMT = 0.75
groups = {
    'single target': {
        'AoE_eff': 1.00,
        'AoE_pct': 0.00
    },
    'eff 25%': {
        'AoE_eff': 0.25,
        'AoE_pct': dMT
    },
    'eff 50%': {
        'AoE_eff': 0.50,
        'AoE_pct': dMT
    },
    'eff 75%': {
        'AoE_eff': 0.75,
        'AoE_pct': dMT
    },
    'eff 100%': {
        'AoE_eff': 1.00,
        'AoE_pct': dMT
    }
}

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=60, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='NPCs',
            range=[0,21],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='encounter multiplier',
            range=[0,10],
            tick0=0, dtick=2,
            minor=dict(tick0=0, dtick=1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

n_max = 20
npcs = list(range(1, n_max+1))
x = list(range(1, n_max+1))
y = [1.0]*n_max

# groups
for group in groups:
    pct = groups[group]['AoE_pct']
    eff = groups[group]['AoE_eff']
    fig.add_trace(go.Scatter(
        x=npcs, 
        y=[encounter_multiplier_approx(pc_count=4, npc_count=n, pc_deaths=0, aoe_pct=pct, aoe_eff=eff) for n in npcs],
        mode='lines', 
        name=group,
        hovertemplate=
            f'<b>{group}</b><br>' + 
            'NPCs %{x:d}<br>'+
            'multiplier %{y:,.1f}' + 
            '<extra></extra>'
    ))

# DMG
group = 'DMG'
fig.add_trace(go.Scatter(
    x=npcs, 
    y=[encounter_multiplier_DMG(4, n) for n in npcs],
    mode='lines', 
    name=group,
    line_color='black',
    hovertemplate=
        f'<b>{group}</b><br>' + 
        'NPCs %{x:d}<br>'+
        'multiplier %{y:,.1f}' + 
        '<extra></extra>'
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name='fig-encounter-multiplier-approx-mixed-target-eff-large')
    #tfb.save_fig_html(fig, format='small', name='fig-encounter-multiplier-approx-mixed-target-eff-small')

In [25]:
# An example encounter diagram where XP earned from AoE damage is marked in red, and damage earned from single target damage is marked in blue
eDPRs = [3, 4, 9]
eHPs  = [3, 6, 7]
eDPR_total = np.sum(eDPRs)
eHP_total = np.sum(eHPs)

damage = [
    {'targets': [0], 'damage': 3, 'efficiency': 1.0, 'color': 'blue'},
    {'targets': [1], 'damage': 6, 'efficiency': 1.0, 'color': 'blue'},
    {'targets': [2], 'damage': 7, 'efficiency': 1.0, 'color': 'blue'}
]

XP_log = create_XP_log(eHPs, eDPRs, damage)

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=15, r=15, b=15, t=15, pad=4),
        title=dict(
            text=r'$\mathrm{PC}\ 1 \rightarrow \mathrm{PC}\ 2 \rightarrow \mathrm{PC}\ 3$',
            font_size=16,
            x=0.5,
            y=0.95,
            xanchor='center',
            yanchor='top',
        ),
        #margin=dict(autoexpand=True),
        dragmode=False,
        xaxis=dict(
            range=[-0.1*eDPR_total, 1.1*eDPR_total],
            #autorange=True,
            mirror=False,
            showgrid=False,
            showline=False,
            zeroline=False,
            showticklabels=False,
            ticks='', minor=dict(ticks=''),
        ),
        yaxis=dict(
            range=[-0.1*eHP_total, 1.1*eHP_total],
            #autorange=True,
            mirror=False,
            showgrid=False,
            showline=False,
            zeroline=False,
            showticklabels=False,
            ticks='', minor=dict(ticks=''),
        ),
    )
)

fig = generate_XP_diagram_plotly(eHPs, eDPRs, XP_log, fig=fig)

# show figure
fig.update_layout(width=600, height=600)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name='fig-xp-encounter-diagram-single-large', style='aspect-ratio: 600/600;')
    #tfb.save_fig_html(fig, format='small', name='fig-xp-encounter-diagram-single-small', style='aspect-ratio: 600/600;')


enc_XP_total = encounter_XP(eDPRs, eHPs)
adj_XP_total = adjusted_XP_total(XP_log)
print('encounter multiplier: {:.2f}'.format(adj_XP_total/enc_XP_total))
eHP_total = sum(eHPs)
dmg_total = damage_total(XP_log)
dmg_overflow = damage_overflow_total(XP_log)
print('percent damage done: {} %'.format(100*dmg_total/eHP_total))
print('percent damage overflow: {} %'.format(100*(dmg_overflow/eHP_total)))

encounter multiplier: 1.97
percent damage done: 100.0 %
percent damage overflow: 0.0 %


In [26]:
# An example encounter diagram where XP earned from AoE damage is marked in red, and damage earned from single target damage is marked in blue
eDPRs = [3, 4, 9]
eHPs  = [3, 6, 7]
eDPR_total = np.sum(eDPRs)
eHP_total = np.sum(eHPs)

damage = [
    {'targets': [0,1,2], 'damage': 3.0, 'efficiency': 1.0, 'color': 'red'},
    {'targets': [1], 'damage': 3.0, 'efficiency': 1.0, 'color': 'blue'},
    {'targets': [2], 'damage': 4.0, 'efficiency': 1.0, 'color': 'blue'}
]

XP_log = create_XP_log(eHPs, eDPRs, damage)

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=15, r=15, b=15, t=15, pad=4),
        #margin=dict(autoexpand=True),
        dragmode=False,
        xaxis=dict(
            range=[-0.1*eDPR_total, 1.1*eDPR_total],
            #autorange=True,
            mirror=False,
            showgrid=False,
            showline=False,
            zeroline=False,
            showticklabels=False,
            ticks='', minor=dict(ticks=''),
        ),
        yaxis=dict(
            range=[-0.1*eHP_total, 1.1*eHP_total],
            #autorange=True,
            mirror=False,
            showgrid=False,
            showline=False,
            zeroline=False,
            showticklabels=False,
            ticks='', minor=dict(ticks=''),
        ),
    )
)

fig = generate_XP_diagram_plotly(eHPs, eDPRs, XP_log, fig=fig)

# show figure
fig.update_layout(width=600, height=600)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name='fig-xp-encounter-diagram-aoe-large', style='aspect-ratio: 600/600;')
    #tfb.save_fig_html(fig, format='small', name='fig-xp-encounter-diagram-aoe-small', style='aspect-ratio: 600/600;')

enc_XP_total = encounter_XP(eDPRs, eHPs)
adj_XP_total = adjusted_XP_total(XP_log)
print('encounter multiplier: {:.2f}'.format(adj_XP_total/enc_XP_total))
eHP_total = sum(eHPs)
dmg_total = damage_total(XP_log)
dmg_overflow = damage_overflow_total(XP_log)
print('percent damage done: {} %'.format(100*dmg_total/eHP_total))
print('percent damage overflow: {} %'.format(100*(dmg_overflow/eHP_total)))

encounter multiplier: 1.28
percent damage done: 100.0 %
percent damage overflow: 0.0 %


In [45]:
# An example encounter diagram where XP earned from AoE damage is marked in red, and damage earned from single target damage is marked in blue
eDPRs = [3, 4, 9]
eHPs  = [3, 6, 7]
eDPR_total = np.sum(eDPRs)
eHP_total = np.sum(eHPs)
damage = [
    {'targets': [0,1], 'damage': 3.0, 'efficiency': 1.0, 'color': 'red'},
    {'targets': [1,2], 'damage': 7.0, 'efficiency': 1.0, 'color': 'blue'}
]

XP_log = create_XP_log(eHPs, eDPRs, damage)

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=15, r=15, b=15, t=15, pad=4),
        #margin=dict(autoexpand=True),
        dragmode=False,
        xaxis=dict(
            range=[-0.1*eDPR_total, 1.1*eDPR_total],
            #autorange=True,
            mirror=False,
            showgrid=False,
            showline=False,
            zeroline=False,
            showticklabels=False,
            ticks='', minor=dict(ticks=''),
        ),
        yaxis=dict(
            range=[-0.1*eHP_total, 1.1*eHP_total],
            #autorange=True,
            mirror=False,
            showgrid=False,
            showline=False,
            zeroline=False,
            showticklabels=False,
            ticks='', minor=dict(ticks=''),
        ),
    )
)

fig = generate_XP_diagram_plotly(eHPs, eDPRs, XP_log, fig=fig)

# show figure
fig.update_layout(width=600, height=600)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name='fig-xp-encounter-diagram-aoe-large', style='aspect-ratio: 600/600;')
    #tfb.save_fig_html(fig, format='small', name='fig-xp-encounter-diagram-aoe-small', style='aspect-ratio: 600/600;')

enc_XP_total = encounter_XP(eDPRs, eHPs)
adj_XP_total = adjusted_XP_total(XP_log)
print('encounter multiplier: {:.2f}'.format(adj_XP_total/enc_XP_total))
eHP_total = sum(eHPs)
dmg_total = damage_total(XP_log)
dmg_overflow = damage_overflow_total(XP_log)
print('percent damage done: {} %'.format(100*dmg_total/eHP_total))
print('percent damage overflow: {} %'.format(100*(dmg_overflow/eHP_total)))

encounter multiplier: 1.28
percent damage done: 100.0 %
percent damage overflow: 25.0 %


In [27]:
# An example encounter diagram where XP earned from AoE damage is marked in red, and damage earned from single target damage is marked in blue
eDPRs = [3, 4, 9]
eHPs  = [3, 6, 7]
eDPR_total = np.sum(eDPRs)
eHP_total = np.sum(eHPs)
damage = [
    {'targets': [1,2], 'damage': 6.0, 'efficiency': 1.0, 'color': 'red'},
    {'targets': [0,2], 'damage': 3.0, 'efficiency': 1.0, 'color': 'blue'}
]

XP_log = create_XP_log(eHPs, eDPRs, damage)

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=15, r=15, b=15, t=15, pad=4),
        #margin=dict(autoexpand=True),
        dragmode=False,
        xaxis=dict(
            range=[-0.1*eDPR_total, 1.1*eDPR_total],
            #autorange=True,
            mirror=False,
            showgrid=False,
            showline=False,
            zeroline=False,
            showticklabels=False,
            ticks='', minor=dict(ticks=''),
        ),
        yaxis=dict(
            range=[-0.1*eHP_total, 1.1*eHP_total],
            #autorange=True,
            mirror=False,
            showgrid=False,
            showline=False,
            zeroline=False,
            showticklabels=False,
            ticks='', minor=dict(ticks=''),
        ),
    )
)

fig = generate_XP_diagram_plotly(eHPs, eDPRs, XP_log, fig=fig)

# show figure
fig.update_layout(width=600, height=600)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name='fig-xp-encounter-diagram-aoe-large', style='aspect-ratio: 600/600;')
    #tfb.save_fig_html(fig, format='small', name='fig-xp-encounter-diagram-aoe-small', style='aspect-ratio: 600/600;')

enc_XP_total = encounter_XP(eDPRs, eHPs)
adj_XP_total = adjusted_XP_total(XP_log)
print('encounter multiplier: {:.2f}'.format(adj_XP_total/enc_XP_total))
eHP_total = sum(eHPs)
dmg_total = damage_total(XP_log)
dmg_overflow = damage_overflow_total(XP_log)
print('percent damage done: {} %'.format(100*dmg_total/eHP_total))
print('percent damage overflow: {} %'.format(100*(dmg_overflow/eHP_total)))

encounter multiplier: 1.19
percent damage done: 100.0 %
percent damage overflow: 12.5 %


In [28]:
# An example encounter diagram where XP earned from AoE damage is marked in red, and damage earned from single target damage is marked in blue
eDPRs = [4, 9, 3]
eHPs  = [6, 7, 3]
eDPR_total = np.sum(eDPRs)
eHP_total = np.sum(eHPs)
damage = [
    {'targets': [0,1], 'damage': 6.0, 'efficiency': 1.0, 'color': 'red'},
    {'targets': [1,2], 'damage': 3.0, 'efficiency': 1.0, 'color': 'blue'}
]

XP_log = create_XP_log(eHPs, eDPRs, damage)

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=15, r=15, b=15, t=15, pad=4),
        #margin=dict(autoexpand=True),
        dragmode=False,
        xaxis=dict(
            range=[-0.1*eDPR_total, 1.1*eDPR_total],
            #autorange=True,
            mirror=False,
            showgrid=False,
            showline=False,
            zeroline=False,
            showticklabels=False,
            ticks='', minor=dict(ticks=''),
        ),
        yaxis=dict(
            range=[-0.1*eHP_total, 1.1*eHP_total],
            #autorange=True,
            mirror=False,
            showgrid=False,
            showline=False,
            zeroline=False,
            showticklabels=False,
            ticks='', minor=dict(ticks=''),
        ),
    )
)

fig = generate_XP_diagram_plotly(eHPs, eDPRs, XP_log, fig=fig)

# show figure
fig.update_layout(width=600, height=600)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name='fig-xp-encounter-diagram-aoe-large', style='aspect-ratio: 600/600;')
    #tfb.save_fig_html(fig, format='small', name='fig-xp-encounter-diagram-aoe-small', style='aspect-ratio: 600/600;')

enc_XP_total = encounter_XP(eDPRs, eHPs)
adj_XP_total = adjusted_XP_total(XP_log)
print('encounter multiplier: {:.2f}'.format(adj_XP_total/enc_XP_total))
eHP_total = sum(eHPs)
dmg_total = damage_total(XP_log)
dmg_overflow = damage_overflow_total(XP_log)
print('percent damage done: {} %'.format(100*dmg_total/eHP_total))
print('percent damage overflow: {} %'.format(100*(dmg_overflow/eHP_total)))

encounter multiplier: 1.19
percent damage done: 100.0 %
percent damage overflow: 12.5 %


In [ ]:
# Calculated encounter multiplier as a function of how much AoE damage is dealt
# by the PCs.
from sympy.utilities.iterables import multiset_permutations

eHPs = np.array([7, 6, 3])
eDPRs  = np.array([9, 4, 3])
eDPR_total = np.sum(eDPRs)
eHP_total = np.sum(eHPs)

nC = len(eHPs)

AoE_pcts = np.linspace(0, 1, 41)
AoE_eff = 1.00

fig = plt.figure(figsize=FSIZE, dpi=FDPI, tight_layout=True)
ax = plt.axes(xlabel='percent of damage from AoE', ylabel='encounter multiplier', ylim=[0.8,2.0])

EMs = np.zeros(len(AoE_pcts))
for z in range(len(AoE_pcts)):
    EMs[z] = encounter_multiplier_ave(eHPs, eDPRs, AoE_pcts[z], AoE_eff)
plt.plot(AoE_pcts, EMs, '--', color='k', label='average')


EMs_ave = np.zeros(len(AoE_pcts))
ittr = np.array(range(nC))
for it in multiset_permutations(ittr):
    for z in range(len(AoE_pcts)):
        EMs[z] = encounter_multiplier_ordered(eHPs[it], eDPRs[it], AoE_pcts[z], AoE_eff)
        EMs_ave[z] += EMs[z]
    plt.plot(AoE_pcts, EMs, '-', color='r', label='order {:}'.format(it), linewidth=0.3)

EMs_ave = np.divide(EMs_ave, len(list(multiset_permutations(ittr))))
plt.plot(AoE_pcts, EMs_ave, '-', color='k', label='average')

ax.grid(which='major', axis='both', alpha=0.4)
#ax.label_outer()

#if SAVEFIGS: plt.savefig('./fig-xp-encounter-diagram.svg', metadata=METADATA)

In [31]:
# Calculated encounter multiplier as a function of how much AoE damage is dealt by the PCs.
def encounter_aHPs(eHPs, AoE_pct):
    # calculates the hit points damaged by AoE abilities after applying AoE damage.
    dmg_AoE = sum(eHPs)*AoE_pct     # total damage dealt by AoE abilities
    n = len(eHPs)                   # number of creatures the AoE damage is spread across
    aHPs = [0]*n                    # list with AoE hit point values
    for i in np.argsort(eHPs):
        aHPs[i] = min(eHPs[i], dmg_AoE/n)
        dmg_AoE -= aHPs[i]
        n -= 1
    return aHPs

def encounter_sHPs(eHPs, AoE_pct):
    # calculates the hit points damaged by single target abilities after applying AoE damage.
    aHPs = encounter_aHPs(eHPs, AoE_pct)
    sHPs = [eHPs[i] - aHPs[i] for i in range(len(eHPs))]
    return sHPs

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=60, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='percent damage from AoE',
            range=[0.0,1.0],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        yaxis=dict(
            title_text='aHP',
            range=[-0.5,7.5],
            tick0=0, dtick=1,
            #minor=dict(tick0=0, dtick=0.5),
        ),
        legend=dict(
            xanchor='right', yanchor='top',
            x=1.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

eHPs = np.array([7, 6, 3])
eDPRs  = np.array([9, 4, 3])
nC = len(eHPs)

nA = 41
AoE_pcts = np.linspace(0, 1, nA)
AoE_eff = 1.00

aHPs = np.zeros((nC,nA))
for z in range(len(AoE_pcts)):
    aHPs[:,z] = encounter_sHPs(eHPs, AoE_pcts[z])

for i in range(nC):
    fig.add_trace(go.Scatter(
        x=AoE_pcts, 
        y=aHPs[i,:],
        mode='lines', 
        line_dash='dash',
        #name=f'{n:d} NPCs - DMG',
        hovertemplate=
            '%{x:.0%} AoE damage<br>'+
            'aHP %{y:,.1f}' + 
            '<extra></extra>'
    ))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name='fig-xp-encounter-diagram-large')
    #tfb.save_fig_html(fig, format='small', name='fig-xp-encounter-diagram-small')

In [ ]:
# depricated functions
def effective_XP_exp(row):
    return 0.25*row['HP Mean']*row['DPR Mean']*np.power(1 + (0.05/0.6), (row['AC'] + row['AB'] - 16))

def effective_XP_linear(row):
    return 0.25*row['HP Mean']*row['DPR Mean']*(1 + (0.05/0.6)*(row['AC'] + row['AB'] - 16))

def CR_to_XP(CR):
    CRs = np.array([0, 1/8, 1/4, 1/2, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 
        14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30])
    XPs = np.array([0, 25, 50, 100, 200, 450, 700, 1100, 1800, 2300, 2900, 3900, 5000, 
        5900, 7200, 8400, 10000, 11500, 13000, 15000, 18000, 20000, 22000, 25000, 
        33000, 41000, 50000, 62000, 75000, 90000, 105000, 120000, 135000, 155000])
    return XPs[np.where(CRs == CR)]

def XP_to_eStats_approx(XP):
    eDPR = np.sqrt(XP/4)
    eHP = 4*eDPR
    return [eHP, eDPR]

def encounter_XPs_self(eHPs, eDPRs):
    # calculates the individual XP for each NPC
    XPs = [0.0]*len(eHPs)
    for i in range(len(eHPs)):
        XPs[i] = 0.25*eHPs[i]*eDPRs[i]
    return XPs

def encounter_XPs_all(eHPs, eDPRs):
    # calculates the XP for every NPC combination
    nC = len(eHPs)
    XPs = np.zeros((nC,nC))
    for i in range(nC):
        for j in range(nC):
            XPs[i,j] = 0.25*eHPs[j]*eDPRs[i]
    return XPs

def encounter_XP(eHPs, eDPRs):
    # calculates the XP total for the encounter
    return 0.25*np.dot(eHPs, eDPRs)

def encounter_aHPs(eHPs, AoE_pct):
    # calculates the hit points damaged by AoE abilities after applying AoE damage.
    dmg_AoE = sum(eHPs)*AoE_pct     # total damage dealt by AoE abilities
    n = len(eHPs)                   # number of creatures the AoE damage is spread across
    aHPs = [0]*n                    # list with AoE hit point values
    for i in np.argsort(eHPs):
        aHPs[i] = min(eHPs[i], dmg_AoE/n)
        dmg_AoE -= aHPs[i]
        n -= 1
    return aHPs

def encounter_sHPs(eHPs, AoE_pct):
    # calculates the hit points damaged by single target abilities after applying AoE damage.
    aHPs = encounter_aHPs(eHPs, AoE_pct)
    sHPs = [eHPs[i] - aHPs[i] for i in range(len(eHPs))]
    return sHPs

def encounter_aXPs_ordered(eHPs, eDPRs, AoE_pct, AoE_eff):
    # calculates the XP components for the encounter after applying AoE damage and then defeating
    # the remaining NPCs in the given order.
    nC = len(eHPs)
    scale = (1-AoE_pct)/1 + AoE_pct/AoE_eff
    sHPs = encounter_sHPs(eHPs, AoE_pct)
    inds = np.argsort([i > 0 for i in sHPs])
    XPs = np.zeros((nC,nC))
    for i in range(nC):
        for j in range(nC):
            ii = inds[i]
            jj = inds[j]
            if i > j:
                XPs[ii,jj] = 0
            elif i < j:
                XPs[ii,jj] = 0.25*sHPs[ii]*eDPRs[jj]*scale
            else:
                XPs[ii,jj] = 0.25*eHPs[ii]*eDPRs[jj]*scale
    return XPs

def encounter_aXP_ordered(eHPs, eDPRs, AoE_pct, AoE_eff):
    # calculates the adjusted XP total for the encounter after applying AoE damage and then defeating the
    # remaining NPCs in the given order.
    aXPs = encounter_aXPs_ordered(eHPs, eDPRs, AoE_pct, AoE_eff)
    aXP = np.sum(aXPs)
    return aXP

def encounter_aXP_ave(eHPs, eDPRs, AoE_pct, AoE_eff):
    # calculates the XP total for the encounter after applying AoE damage and then averaging across 
    # all possible orders for defeating the remaining NPCs.
    sHPs = encounter_sHPs(eHPs, AoE_pct)
    scale = (1-AoE_pct)/1 + AoE_pct/AoE_eff
    XP = 0
    for i in range(len(eDPRs)):
        for j in range(len(eHPs)):
            if i == j:
                eDPR = eDPRs[i]
                eHP  = eHPs[j]
            else:
                eDPR = eDPRs[i] if sHPs[i] > 0 else 0
                eHP  = sHPs[j]/2
            XP += 0.25*eHP*eDPR*scale
    return XP

def encounter_multiplier_ordered(eHPs, eDPRs, AoE_pct, AoE_eff):
    # calculates the encounter multiplier for the encounter after applying AoE damage and then defeating the
    # remaining NPCs in the given order.
    XP = encounter_XP(eHPs, eDPRs)
    aXP = encounter_aXP_ordered(eHPs, eDPRs, AoE_pct, AoE_eff)
    return aXP/XP

def encounter_multiplier_ave(eHPs, eDPRs, AoE_pct, AoE_eff):
    # calculates the encounter multiplier for the encounter after applying AoE damage and then averaging across 
    # all possible orders for defeating the remaining NPCs.
    XP = encounter_XP(eHPs, eDPRs)
    aXP = encounter_aXP_ave(eHPs, eDPRs, AoE_pct, AoE_eff)
    return aXP/XP

def encounter_multiplier_DMG(pc_count, npc_count):
    """Returns the encounter multiplier given by the DMG
    pc_count -- number of PCs in the encounter
    npc_count -- number of NPCs in the encounter
    """
    n_array = np.asarray([1,2,3,7,11,15])
    m_array = np.asarray([0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 4.0, 5.0])
    i = 1 + n_array[n_array <= max(npc_count,1)].argmax()
    if pc_count >= 6:
        i -= 1
    elif pc_count <= 2:
        i += 1
    return m_array[i]

def encounter_multiplier_approx(pc_count=4, npc_count=1, pc_deaths=0, aoe_pct=0, aoe_eff=1.0):
    """Returns the approximate encounter multiplier
    pc_count -- number of PCs in the encounter
    npc_count -- number of NPCs in the encounter
    pc_deaths -- number of PCs expected to die during encounter
    aoe_pct -- decimal percent of damage done to each NPC using AoE effects
    AoE_eff -- ratio of AoE DPR per target to single target DPR
    """
    pd = min(pc_deaths, pc_count)
    em = (npc_count + 1)*(1 - aoe_pct)/2 + aoe_pct/aoe_eff
    em *= 8.0*pc_count/(2*pc_count*pc_count - 2*pd*pc_count + pd*(pd + 1))
    return em

def adjusted_XP_total(XP_log):
    return np.sum([x['XP'] for x in XP_log])

def encounter_weights(eHPs, eDPRs, XP_log):
    max_XPs = encounter_XPs_all(eHPs, eDPRs)
    enc_XPs = np.zeros(max_XPs.shape)
    for x in XP_log:
        enc_XPs[x['ally'], x['target']] += x['XP']
    enc_Ws = np.divide(enc_XPs, max_XPs)
    return enc_Ws

def damage_total(XP_log):
    dmg = 0
    for x in XP_log:
        if x['target'] == x['ally']:
            dmg += x['eHP_delta']
    return dmg

def damage_overflow_total(XP_log):
    dmg = 0
    for x in XP_log:
        if x['target'] == x['ally']:
            dmg += x['damage'] - x['eHP_delta']
    return dmg



In [ ]:
# more depricated functions
def encounter_XP_grid_approx(XP):
    # calculates the XP for every NPC combination
    nC = len(XP)
    XP_grid = np.zeros((nC,nC))
    for i in range(nC):
        for j in range(nC):
            XP_grid[i,j] = np.sqrt(XP[i]*XP[j])
    return XP_grid

def encounter_weight_grid_mixed_random(XP, AoE_pct, AoE_eff):
    nC = len(XP)
    w = np.zeros((nC,nC))
    for i in range(nC):
        for j in range(nC):
            if i == j:
                w[i,j] = 1 - AoE_pct[i] + AoE_pct[i]/AoE_eff
            if AoE_pct[j] == 1:
                w[i,j] = 0
            else:
                w[i,j] = 0.5*(1 - AoE_pct[i])
    return w

def encounter_XP_total_DMG(XP):
    return np.sum(XP)*encounter_multiplier_DMG(4, len(XP))

def encounter_XP_total(XP_grid, weight_grid):
    return np.sum(np.multiply(weight_grid, XP_grid))

def XP_total(XP):
    return np.sum(XP)



In [ ]:
def generate_XP_diagram_matplotlib(ax, eHPs, eDPRs, XP_log, title='', fontsize=10):
    nC = len(eHPs)
    eHP_mins = [sum(eHPs[0:i]) for i in range(len(eHPs))]
    eDPR_mins = [sum(eDPRs[0:i]) for i in range(len(eDPRs))]
    eHP_max = sum(eHPs)
    eDPR_max = sum(eDPRs)

    # initialize figure
    ax.add_patch(plt.Rectangle((0,0), eDPR_max, eHP_max, fc='white', ec='none', alpha=1.0))

    # apply damage and color XP boxes accordingly
    for x in XP_log:
        ax.add_patch(plt.Rectangle((x['eDPR_min'], x['eHP_min']), x['eDPR_delta'], x['eHP_delta'], 
            fc=x['color'], ec='none', alpha=0.3))

    # create XP box borders
    for i in range(nC):
        for j in range(nC):
            if i == j:
                ls = '-'
            else:
                ls = '-'
            ax.add_patch(plt.Rectangle((eDPR_mins[i], eHP_mins[j]), eDPRs[i], eHPs[j], fc='none', ec='black', linestyle=ls))

    # create XP box labels
    plt.rcParams['text.usetex'] = True
    for i in range(nC):
        for j in range(nC):
            ax.text(eDPR_mins[i] + 0.5*eDPRs[i], eHP_mins[j] + 0.5*eHPs[j], 
                r'$\mathit{{XP}}_{{{:},{:}}}$'.format(j+1,i+1), fontsize=fontsize, ha='center')

    # create axes labels
    for i in range(nC):
        ax.text(eDPR_mins[i] + 0.5*eDPRs[i], -(0.7/16)*eHP_max, 
            r'$\mathit{{eDPR}}_{{{:}}}$'.format(i+1), fontsize=fontsize, ha='center', va='center')
        ax.text(-(0.7/16)*eDPR_max, (eHP_mins[i] + 0.5*eHPs[i]), 
            r'$\mathit{{eHP}}_{{{:}}}$'.format(i+1), fontsize=fontsize, ha='center', va='center', rotation='vertical')

    # create title
    ax.text(0.5*eDPR_max, eHP_max*(1 + (0.7/16)), title, fontsize=fontsize, ha='center', va='center')
    plt.rcParams['text.usetex'] = False

    plt.axis('off')
    plt.axis('scaled')

    return ax